# Specialized Attention Tutorial

- Last Update : 18 / 10 / 2018
- Author : Yann Dubois

In [ ]:
import sys

# you have to be on the branch of machien-tasks that use `spattn` not `seq2seq`
sys.path.append("../machine-tasks/tasks")

from reporter import generate_multireport, dev_predict

OUTPUT_DIR = "tutorial/"

%matplotlib inline

/Users/yanndubois/venvs/spattn/lib/python3.7/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))
/Users/yanndubois/venvs/spattn/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/yanndubois/venvs/spattn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Setup

0. Clone `spattn` () and my `machine-tasks` ()
1. Activate your virtual environment
2. Install `spattn` (the smaller version of the current `machine`). Using `pip install -e /path/to/spattn`
3. Install jupyter notebook `pip install jupyter`
4. Run jupyter notebook `jupyter notebook`
5. Run all the following code (do this before reading the rest, such that the example already starts running : it should take 20 min hour)

## Disclaimers

 1. This **is not the main library**, it is a **subset** of the main code that contains the most important hyperparameters. I removed some hyperparameters like : biderictional rnns, multilayer rnn, post-attention, attentive gaussian, hard guidance, many of the parameters I tried that I don't think we will be using on the long run ... That might improve the results but are not necessary to understand the library / research. this will hopefully help you understand quicker the meat of the work. Note that this library is still pretty large.
 
 2. Because of number 1/, there might be some docstrings that I forgot to remove (i.e documentation for parameters that you don't have). If this is the case you **should** remove them and push the updated version on github (indeed I think that this version is actually the one you should be aiming to make self-attentive) . Similarly there might be some functions that simply contain a single line of code because I removed the rest. I kept those functions so that the general pipeline is still the same as the main library. Finally I haven't tested all parameters in this smaller version : if you see minor bugs then change them and if you really can't find them let me know and I can probably change it (or at least tell you what to change).
 
 3. **THERE's ACTUALLY A SMALL BUG IN THIS LIBRARY WHICH IMPACTS HOW THE CONFUSER WORK**. I will debug it when I have time. But keep in mind that the **confusers** don't work well. I.e to use both types of attention you should either try normally and see that it will (probably) not work or you can try using `"is_reg_pos_perc"` in "the reguarization (not tested but should work decently well).

## Structure

The library is separated into `machine-tasks` which contains the tasks and `machine` (here `specialized-attention-mini` as it is a subset) which contains the code. In the near future `machine` should be splitted up into `machine` which will be the high level code, which we shouldn't modify and `spattn` (name to change) which will correspond to our research.

- `machine-tasks` contains the tasks we have been working with. All the ones I generated, come from `machine-tasks/LongLookupTables/make_lookup_tables.py`. These are variants on the lookup tables task, that I thought we should look at.
- `machine-tasks` also contains a `tasks` directory, where I wrote some helper functions to generate the reports. It's not the cleanest code, but it's not the most import code to clean as long as it works. This defines the main function you will ba calling : `generate_multireport`.
- `generate_multireport` instantiates a `seq2seq` model through `train`and `get_seq2seq` of `main.py`. These 2 functions have all the high level docstrings and you should check those to understand the high level meaning of a hyperparameter.
- `seq2seq` has an `encoder` and a `decoder` attribute. Those define most of the method we are proposing. this is especially true for the `decoder`as it contains the `ContentAttention`, the `PositioningAttention`, as well as an `AttentionMixer`.
- Just as you asked, or I wrote `Contact: Yann Dubois` at the top of the modules of `machine` that I have written or nearly completely modified. For the code in `machine-tasks` it's all me although I haven't written my name.

I normally run my code from `machine-tasks` and installed `machine` with `pip install -e /path/to/machine`


## Notes

- While you are getting your hands dirty with the code, it is important that you modify errors in the code or docstrings / fill in missing docstrings (there are probably quite a few in the smaller version of the code that you have). Indeed this will be given in case an other student works with you, and making his life easier would make yours easier. Additionaly if you work on this for more than a few weeks, your future self will be very happy if you make those changes now rather than having to find errors / documentation.

- much of the code seems a bit ad-hoc. This is because I have built upon the machine library and used to maitain everything such that it would not brake their Attentive Guidance work. That meant a lot of tricks, that you shouldn't be using. When modifying the library to work with self-attention, you should focus on our current method and be modular for future improvements : but not for others work. 

# Example

In [ ]:
sys.path.append("../tasks")

from reporter import *

OUTPUT_DIR = "tutorial-reports/"

%matplotlib inline

In [ ]:
%%time

# some test examples for different dataset
srcs = {"Long Lookup Table":"001 t1 t6 t5 t5 t6 t6 t3 .", 
         "Long Lookup Table Reverse":"t3 t6 t1 t3 t3 t3 t4 101 .", 
         "Noisy Long Lookup Table Single":"100 t5 t4 t6 ! t2 t2 t3 t3 t4 t6 t2 ."}

# run the default model on the Long Lookup Table dataset
kwargs = {} # here is where you define all the hyperparameters
models, others = generate_multireport([get_task("long lookup", is_small=True)], # define the tasks you want to look at
                                       OUTPUT_DIR,  # where to save the model
                                       name="example",  # first word of the name of the model
                                       k=1,  # run only with a single random seed. For getting publishable results this has to be bigger
                                       **kwargs) 

# this is not necessary : I just prefer seeing a small example in the notebook beofre opening the report
for k, v in others.items():
    to_show[v['task_path']] = srcs[k]
    pred, other, test = dev_predict(v['task_path'], srcs[k])  # `dev_predict` predicts but also returns some interpretable variables in `test`

Now open the report which should be in "tutorial-reports/example_/Long Lookup Table/report.pdf". And try understanding the output using the next section.

If something is still not clear, feel free to slack me.

## Report Interpretation



*Nota Bene: this is a random report I had : yours will be different.*

### Names

The naming pattern is automatically generated given the `**kwargs` you gave to `generate_multireport`. For the code (and up-to-date) information about how it is generated, please look at `machine-tasks/tasks/reporter.py/_namer`

The general concept is that each `kwarg` will be be shown as `key_value` in alphabetic order. For `bool`'s I drop the value, and simply us `is_key` or `not_key`.

### Model

All the classes that enherit from `torch.nn.Module` should enherit from `spattn.util.base.Module` instead. This does multiple things including forcing you to define `extra_repr` so that the model can be printed in a nice way.

The output of this printing procedure can be seen on this page :

![Example Report Image](img/example_report_model.png)

### Training and Validation Losses

This is self explanatory. Note that the number of epochs is not always equal to `epochs` because of early stopping.

![Example Report Losses](img/example_report_losses.png)

*Nota Bene : this specific plot is really not looking good, normally it will be similar to f(x) = 1/x. Yours should be like this (this plot is due to specific hyperparameters).*

### Metrics

This will plot all the final `loss` and the final `metrics` that were given to the report (current default : `["word accuracy", "sequence accuracy", "final target accuracy"]`). Those are evaluated on the training / testing / validation tests that were defined in `machine-tasks/tasks/tasks.py/get_task`.

![Example Report Metrics](img/example_report_metrics.png)

For the long lookup table task and its variation, we mostly care about : "train", "validation", "heldout_inputs", "heldout_compositions", "longer_seen_1", ..., "longer_seen_n", As these are the only ones that can be perfectly solved using hard attention.

If `k>1` then the plot will show the average over `k` runs and the 9% % bootstrap confidence interval.

### Variation of Intepretable Variables during Training

The next 3 pages show how some interpretable changed during training time. Those are only variables that are independent of the sequence length. If they were dependent then I avergaed over all the length of the sequence.

The first page simply shows a bunch of those (note that the ones shown will depend on the hyperparameters you chose).

![Example Report Interptable Variables](img/example_report_interpretable_variables.png)

The second page (only if using position attention) show how the weight and the value of each building block change during training.

![Example Report Building Block](img/example_report_bulding_block.png)

The third page show how the raw values of the regularization losses change with the numebr of training epochs.

![Example Report Raw Regularization](img/example_report_raw_regularization.png)

The 4th page show how the the actual value of the regularization losses we are using (i.e weighted). These are shown in percentage of the total loss.

![Example Report Weighted Regularization](img/example_report_weighted_regularization.png)

### Examples of Attention Patterns

In the final pages, I sample `n_attn_plots` (current default : 3) from each training / testing / validation set, and I plot the attention for the content, position, and final attention (if those are given). Additionaly, when using position attention, I also show a table that shows some interptable variables (mu, sigma, the values of the building block weights, the confidence ... Please see `machine-tasks/tasks/visualizer/AttentionVisualizer.positional_table_labels` for the updated list).

![Example Report Attention](img/example_report_attention.png)

# Exercises

The goal of these simple "exercises":

- is simply to give you a feeling of how the code works. The results are not the bests and it will be mostly a task of reading the docstrings and understanding how the first layer of the library works. Because of this you might want to decrease the maximum number of epoch if it th default 100 step is too much.

- to give you a sense of what is currently working well and what has to improve / isn't working yet.

- give you a sense of the datasets I have been working with.

It is importnat that you think about what will happen before running the code, and the write-up (just a few notes so that we can talk about it) once you interpreted the results. There are often no correct answers, but I want to know what your interpretation is.

## Run Long Lookup Table with different attentions

### Run the model without positioning attention

#### Hypothesis
*Fill In*

####  Results + Interpretatio 
*Fill In*

In [ ]:
# code

### Run the model without content attention

#### Hypothesis
*Fill In*

####  Results + Interpretation
*Fill In*

In [ ]:
# code

## Run default model on different datasets

### Long Lookup Table Reverse

#### Hypothesis
*Fill In*

#### Results + Interpretation
*Fill In*

### Long Lookup Table Single

#### Hypothesis
*Fill In*

#### Results + Interpretation
*Fill In*

### Noisy Lookup Table Single

#### Hypothesis
*Fill In*

#### Results + Interpretation
*Fill In*

In [ ]:
# code

## Run simple Position Attention model on different datasets

Deactivate all the hyperparameters that seem to only help the position attention but are not core to it. Namely **rounder_weights**, all the **regularizations**, the **content attention**. For both datasets explain which hyperparameter you think had which effect.

### Long Lookup Table Reverse

#### Hypothesis
*Fill In*

#### Results + Interpretation
*Fill In*

### Long Lookup Table Single

#### Hypothesis
*Fill In*

#### Results + Interpretation
*Fill In*

In [ ]:
# Code

## Run baseline model

In order to understand how the model improves compared to the the basline model, let's have a look at when we don't use positioning attention and don't use key / value / query / highway.

Although this is possible in the main code, I removed the possibility to do so in this subset of the code, so that you could change a few lines of code yourself to use the baseline model. This should be very straightforward, but give you a feel of the code.

### Baseline

#### Hypothesis
*Fill In*

#### Results + Interpretation
*Fill In*

In [ ]:
# code

# Contributing

Please read the **README.md**. Note that I'm always open to change how we do if there's good reason (nothing is set in stone :) ).